Tasks: 

1. Get the DFs of the PBM
2. Given a sequence, get the TFs that bind to it well
3. Get the points of interest in the MPRA DF and implement a running window.
4. Think how to present the differences between Ancestral and Derived. 

In [1]:
# First let's import the libraries we will use:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from ortals_code.TF_review_in_seq import *
import ortals_code.TF_review_in_seq as ortals
from functools import reduce
import utils_matanya as um

In [7]:
# Useful constants
DATA_TF_PATH = "/home/labs/davidgo/ortalh/TF_analysis_data/data_TF_for_analysis"
MY_DATA_PATH = "/home/labs/davidgo/matanyaw/data"
MPRA_DATA = "/home/labs/davidgo/Collaboration/humanMPRA/chondrocytes/comparative_analysis_combined/humanMPRA_with_seq_final2.csv"

                

In [4]:
all_8mer_pbm_df = pd.read_csv(os.path.join(MY_DATA_PATH, "all_8mer_pbm_multiindex.csv"), index_col=0, header=[0,1])
all_8mer_pbm_df

E-score                                                          \
              ARX Ahctf1_mus_musculus Alx3_mus_musculus Alx4_mus_musculus   
8-mer                                                                       
AAAAAAAA -0.26798             0.48566           0.28589           0.23745   
AAAAAAAC -0.37748             0.44921           0.13567           0.14823   
AAAAAAAG -0.34844             0.44895           0.18368           0.10684   
AAAAAAAT -0.18697             0.48716           0.29545           0.24582   
AAAAAACA -0.26721             0.42078           0.17853           0.06920   
...           ...                 ...               ...               ...   
TTTCCAAA -0.15100             0.32919           0.08317           0.02217   
TTTCGAAA -0.26713             0.24944           0.01566          -0.00269   
TTTGAAAA -0.24814             0.34429           0.27189           0.19366   
TTTGCAAA -0.08510             0.24837           0.11334           0.07307   
TTTTAAAA -0.39655             0.45606           0.22733           0.16690   

                                                                  \
         Ar_mus_musculus Arid3a_mus_musculus Arid5a_mus_musculus   
8-mer                                                              
AAAAAAAA         0.12735         3227.916623         4423.780237   
AAAAAAAC         0.09505         2533.853384         3396.074517   
AAAAAAAG         0.05979         3050.847409         2999.839672   
AAAAAAAT         0.06019         3623.556723         3157.582398   
AAAAAACA         0.21409         2915.334214         3319.464399   
...                  ...                 ...                 ...   
TTTCCAAA        -0.02246         2538.460427         2784.739025   
TTTCGAAA         0.01719         2802.119103         2670.396223   
TTTGAAAA         0.21231         2887.502101         3305.476256   
TTTGCAAA         0.01220         2282.289378         2147.510480   
TTTTAAAA         0.10100         2507.230014         3231.472131   

                                                        ...  \
         Ascl2_mus_musculus Atf3_mus_musculus   BCL11A  ...   
8-mer                                                   ...   
AAAAAAAA        4424.472905           0.29716  0.03392  ...   
AAAAAAAC        3619.284343           0.19871 -0.07074  ...   
AAAAAAAG        3118.896750           0.35041 -0.15798  ...   
AAAAAAAT        3588.531769           0.34566 -0.18789  ...   
AAAAAACA        3394.106587           0.25221 -0.04663  ...   
...                     ...               ...      ...  ...   
TTTCCAAA        2715.747593           0.13544 -0.03857  ...   
TTTCGAAA        2830.764797           0.21747 -0.18407  ...   
TTTGAAAA        3004.966035           0.20149 -0.13126  ...   
TTTGCAAA        2968.135781           0.18509 -0.14111  ...   
TTTTAAAA        3525.396054           0.32578  0.02176  ...   

                     Z-score                                             \
         TFAP2A_mus_musculus Tbx2_mus_musculus Tef_mus_musculus    VAX2   
8-mer                                                                     
AAAAAAAA              0.8049           -0.2041           0.4316  0.4226   
AAAAAAAC             -0.3715            0.7683           1.4607  0.1454   
AAAAAAAG              1.5246            0.8301           0.6015 -0.5753   
AAAAAAAT              1.7388            1.1866           1.4478  1.1230   
AAAAAACA             -1.8648            1.1769           1.4901  0.5406   
...                      ...               ...              ...     ...   
TTTCCAAA              1.2079            1.0096           0.2931  1.0831   
TTTCGAAA              0.1766            0.7694           1.6737  1.4748   
TTTGAAAA              1.3062            1.7171           1.8915  0.9733   
TTTGCAAA              0.7302            1.9076           0.5129  2.8273   
TTTTAAAA              1.5928            0.6025           1.5853  6.2740   

                                            

In [50]:
escore_df, median_df, zscore_df = all_8mer_pbm_df['E-score'], all_8mer_pbm_df['Median'], all_8mer_pbm_df['Z-score']
zscore_df

,ARX,Ahctf1_mus_musculus,Alx3_mus_musculus,Alx4_mus_musculus,Ar_mus_musculus,Arid3a_mus_musculus,Arid5a_mus_musculus,Ascl2_mus_musculus,Atf3_mus_musculus,BCL11A,...,TFAP2A_mus_musculus,Tbx2_mus_musculus,Tef_mus_musculus,VAX2,VENTX,VSX1,VSX2,WT1,ZNF200,ZNF655
8-mer,,,,,,,,,,,,,,,,,,,,,
AAAAAAAA,-2.5642,9.0057,2.3401,1.7044,1.1648,2.423686,6.037755,5.100156,2.0726,0.7592,...,0.8049,-0.2041,0.4316,0.4226,-4.0103,1.1738,1.9972,1.3045,1.6922,0.1415
AAAAAAAC,-2.1900,7.2302,1.3416,1.4467,1.1031,-0.021227,3.549164,2.741533,1.5885,-0.9601,...,-0.3715,0.7683,1.4607,0.1454,-2.8133,0.2560,1.8681,0.8530,1.7340,0.0316
AAAAAAAG,-2.9855,6.1877,1.3386,1.0929,-0.0242,1.853928,2.381357,0.994396,2.8981,-0.7073,...,1.5246,0.8301,0.6015,-0.5753,-3.8743,-0.0920,0.8050,1.4324,1.8098,0.5059
AAAAAAAT,-1.8606,8.8501,2.1334,2.3737,0.1240,3.591318,2.863760,2.641339,2.4990,-0.6869,...,1.7388,1.1866,1.4478,1.1230,-1.8313,1.9065,2.6485,1.1602,1.4194,0.9966
AAAAAACA,-2.2716,5.3682,1.6949,1.1549,1.7486,1.395083,3.334386,1.987293,1.7003,-0.2942,...,-1.8648,1.1769,1.4901,0.5406,-1.8409,0.2393,1.8914,1.5113,1.7164,0.2976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTCCAAA,-0.2410,2.9411,0.5581,0.2975,-0.0945,-0.002882,1.680976,-0.630805,0.9508,-0.2414,...,1.2079,1.0096,0.2931,1.0831,-0.7304,1.0992,1.2829,0.7699,0.8522,0.6617
TTTCGAAA,-2.3461,1.3296,0.8202,-0.0643,1.5874,0.995079,1.286309,-0.143761,1.7645,-1.5172,...,0.1766,0.7694,1.6737,1.4748,-2.8707,-0.2231,2.3837,0.7933,0.0525,0.4264
TTTGAAAA,-1.9434,3.5111,1.6207,1.2021,1.5820,1.298207,3.294636,0.557449,1.6849,0.0998,...,1.3062,1.7171,1.8915,0.9733,-3.1528,1.5440,2.3192,1.2091,1.5615,0.6381


This is how I get the binding info for an 8mer:

In [6]:
seq = "AACCAATT"
reverse_seq = um.get_min_seq(seq)

escore_df.loc[reverse_seq]

ARX                    0.43086
Ahctf1_mus_musculus    0.29898
Alx3_mus_musculus      0.42600
Alx4_mus_musculus      0.43650
Ar_mus_musculus       -0.08729
                        ...   
VSX1                   0.44190
VSX2                   0.31273
WT1                    0.06924
ZNF200                 0.06756
ZNF655                 0.01405
Name: AACCAATT, Length: 120, dtype: float64

# Dealing with MPRA data

In [ ]:
full_mpra_df = pd.read_csv(MPRA_DATA)
full_mpra_df

,oligo,RNA_DNA_ratio_log2_ancestral,RNA_DNA_ratio_log2_derived,DNA_counts_raw_ancestral,DNA_counts_raw_derived,RNA_counts_raw_ancestral,RNA_counts_raw_derived,barcode_count_ancestral,barcode_count_derived,alpha_activity_estimate_ancestral,...,normalized_activity_estimate_derived,activity_fdr_ancestral,activity_fdr_derived,activity_ancestral,activity_derived,logFC_derived_vs_ancestral,differential_activity_fdr,differential_activity,sequence_ancestral,sequence_derived
0,seq_100000_chr8:30170404-30170673_SCREEN_a3_L1,0.305667,0.572193,870.0,933.0,3400.0,4386.0,75.0,98.0,2.912881,...,6.972534,0.000003,4.327965e-11,active,active,0.062453,0.634114,False,gtctttatcagcagcatgaaaatgaactaatatacctcctttcttt...,gtctttatcagcagcatgaaaatgaactaatatacctcctttcttt...
1,seq_100001_chr11:26270190-26270459_SCREEN_a3_L1,-0.847603,-0.810681,1120.0,1105.0,1967.0,1991.0,90.0,81.0,1.436428,...,-0.465257,0.852420,8.280452e-01,non_active,non_active,NaN,NaN,NaN,tttattcagacacaatgattctgctgaatgttcaaagattttcttt...,tttattcagacacaatgattctgctgaatgttcaaagattttcttt...
2,seq_100002_chr19:42985016-42985285_SCREEN_a3_L1,0.362402,0.122223,211.0,446.0,860.0,1536.0,33.0,55.0,2.768096,...,2.245034,0.000046,7.787675e-02,active,non_active,-0.271145,0.063988,False,ccctgctatataaactccccagtttcagtctgtcagagattgatct...,ccctgctatataaactccccagtttcagtctgtcagagattgatct...
3,seq_100003_chr8:34830463-34830732_SCREEN_a3_L1,-0.714446,-0.874729,307.0,764.0,592.0,1317.0,29.0,51.0,1.629875,...,-0.777659,0.705492,8.804929e-01,non_active,non_active,NaN,NaN,NaN,agtacaacagaaaagcaaatcatgacgagaatgaagcctgaactct...,agtacaacagaaaagcaaatcatgacgagaatgaagcctgaactct...
4,seq_100004_chr8:17086465-17086734_SCREEN_a3_L1,-1.092105,-0.612584,358.0,271.0,531.0,561.0,14.0,13.0,1.296493,...,0.890924,0.932396,4.841857e-01,non_active,non_active,NaN,NaN,NaN,tttactttgtgaagtatatccttcggaataaatgcattcctggaaa...,tttactttgtgaagtatatccttcggaataaatgcattcctggaaa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355886,seq_99995_chr1:232513928-232514197_SCREEN_a3_L1,-0.842769,-0.549248,326.0,364.0,575.0,787.0,11.0,20.0,1.572049,...,1.192519,0.755333,3.763739e-01,non_active,non_active,NaN,NaN,NaN,agagtgagaccctgtctcaaaaaaaaaaaaaaaatagtttaagtaa...,agagtgagaccctgtctcaaaaaaaaaaaaaaaatagtttaagtaa...
355887,seq_99997_chr8:98081035-98081304_SCREEN_a3_L1,-0.469455,-0.323055,1767.0,1162.0,4033.0,2936.0,120.0,88.0,1.817734,...,1.816562,0.494230,1.707516e-01,non_active,non_active,NaN,NaN,NaN,ggttaaatgacttgcccaagactatactgcccttagggcaggagtc...,ggttaaatgacttgcccaagactatactgcccttagggcaggagtc...
355888,seq_99998_chr3:135890202-135890471_SCREEN_a3_L1,-0.598124,-0.731697,1298.0,1762.0,2710.0,3353.0,89.0,105.0,1.692378,...,-0.077829,0.644568,7.539551e-01,non_active,non_active,NaN,NaN,NaN,aaagctgaatgactccttagggttgtcctatgctgggcccaaggag...,aaagctgaatgactccttagggttgtcctatgctgggcccaaggag...
355889,seq_99999_chr4:182061380-182061649_SCREEN_a3_L1,-0.759305,-0.713852,695.0,1163.0,1298.0,2241.0,55.0,77.0,1.539958,...,-0.411293,0.780191,8.186231e-01,non_active,non_active,NaN,NaN,NaN,ttcattcacacattttaattctttattctcagactgtgcaatcatt...,ttcattcacacattttaattctttattctcagactgtgcaatcatt...


In [ ]:
mpra_df = full_mpra_df[full_mpra_df['differential_activity'] == True]
mpra_df

,oligo,RNA_DNA_ratio_log2_ancestral,RNA_DNA_ratio_log2_derived,DNA_counts_raw_ancestral,DNA_counts_raw_derived,RNA_counts_raw_ancestral,RNA_counts_raw_derived,barcode_count_ancestral,barcode_count_derived,alpha_activity_estimate_ancestral,...,normalized_activity_estimate_derived,activity_fdr_ancestral,activity_fdr_derived,activity_ancestral,activity_derived,logFC_derived_vs_ancestral,differential_activity_fdr,differential_activity,sequence_ancestral,sequence_derived
37,seq_100038_chr6:4358790-4359059_SCREEN_a3_L1,0.863731,0.991378,441.0,1310.0,2540.0,8233.0,30.0,89.0,4.442910,...,13.975102,1.434529e-26,9.453240e-43,active,active,0.299749,1.584436e-02,True,cagggatgggcacacctggccggctgaggggagcctacagccaggc...,cagggatgggcacacctggccggctgaggggagcctacagccaggc...
63,seq_100065_chr7:138979123-138979392_SCREEN_a3_L1,-0.727351,-0.051537,1894.0,2545.0,3615.0,7760.0,134.0,152.0,1.626701,...,3.538402,7.083968e-01,2.219732e-03,non_active,active,0.316918,4.850811e-06,True,tccattttaccaatcacgaaactgaatcagagagctaagcaacttg...,tccattttaccaatcacgaaactgaatcagagagctaagcaacttg...
68,seq_100070_chr7:79861027-79861296_SCREEN_a3_L1,0.033310,-0.317820,1085.0,549.0,3510.0,1393.0,71.0,54.0,2.536598,...,1.484645,1.606845e-03,2.730638e-01,active,non_active,-0.275615,1.638135e-02,True,ctgtttattgttatactcaagagcacttggtattgttaaagtcgta...,ctgtttattgttatactcaagagcacttggtattgttaaagtcgta...
73,seq_100075_chr16:54376420-54376689_SCREEN_a3_L1,-0.063909,0.348923,1121.0,1485.0,3390.0,5978.0,67.0,94.0,2.370884,...,5.932758,1.232513e-02,3.268023e-08,active,active,0.318055,1.342002e-03,True,aggaaggaggatctctgtgtgtgtgggtcgcccagcctcctgccag...,aggaaggaggatctctgtgtgtgtgggtcgcccagcctcctgccag...
88,seq_100090_chr20:31380149-31380418_SCREEN_a3_L1,0.724860,0.382225,1558.0,2152.0,8139.0,8864.0,97.0,106.0,4.313777,...,6.817569,2.892071e-24,1.244647e-10,active,active,-0.281553,6.067254e-04,True,atgtctgccttttgagtgggaggtgaccactgacattccctagggg...,atgtctgccttttgagtgggaggtgaccactgacattccctagggg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355815,seq_99921_chr11:34262393-34262662_SCREEN_a3_L1,-0.509656,-0.119381,1127.0,784.0,2502.0,2282.0,76.0,69.0,1.829854,...,2.631194,4.778543e-01,3.238074e-02,non_active,active,0.286606,3.722563e-03,True,tggcctcccaaagtgctgggattacaggtgtgagccaccgtacctg...,tggcctcccaaagtgctgggattacaggtgtgagccaccgtacctg...
355824,seq_99930_chr10:128329049-128329318_SCREEN_a3_L1,1.000198,-0.163459,660.0,702.0,4176.0,1982.0,33.0,46.0,4.948771,...,2.531735,1.244198e-36,4.122374e-02,active,active,-0.770237,1.065995e-08,True,ccctgagcttggaaacatagagatgagaaaataagttccctacact...,ccctgagcttggaaacatagagatgagaaaataagttccctacact...
355857,seq_99966_chr21:35967796-35968065_SCREEN_a3_L1,-0.030951,-0.383297,2474.0,1475.0,7652.0,3574.0,182.0,101.0,2.453788,...,1.279978,4.690473e-03,3.446261e-01,active,non_active,-0.230129,2.304378e-03,True,gtgtacagacccatctattcatcatagacttcactgtctgggaatg...,gtgtacagacccatctattcatcatagacttcactgtctgggaatg...
355864,seq_99973_chr14:22846595-22846864_SCREEN_a3_L1,0.667980,-0.637902,359.0,429.0,1806.0,872.0,75.0,81.0,2.867773,...,-0.543097,7.781080e-06,8.414780e-01,active,non_active,-0.305473,1.013274e-02,True,gtcaagttggaacttagattctttttcttgcggaagtgggaggcat...,gtcaagttggaacttagattctttttcttgcggaagtgggaggcat...


In [29]:
seq_anc = mpra_df.loc[0, 'sequence_ancestral']
seq_der = mpra_df.loc[0, 'sequence_derived']

print(seq_anc)
# print(seq_der)

def running_window(seq, window_size, start_idx, end_idx):
    return [seq[i:i+window_size] for i in range(start_idx, end_idx-window_size+1)]

windows = running_window(seq_anc, window_size=8, start_idx=0, end_idx=len(seq_anc))
for w in windows:
    print(w)

len(windows)

gtctttatcagcagcatgaaaatgaactaatatacctcctttctttataaattactcaggtagtatctttatagcagtgtgagaaaggactaataaatgctctaaagaaaggatgaatcatcattaaataagaggTtgggtgaggggtggagttaatggtaatgcccttgatgctactgaatcagctgggtttcagactattcttctgtcccgaggtctccaaggcctgctcacagtgcagtcatgttttgggcttcaggacctccagcc
gtctttat
tctttatc
ctttatca
tttatcag
ttatcagc
tatcagca
atcagcag
tcagcagc
cagcagca
agcagcat
gcagcatg
cagcatga
agcatgaa
gcatgaaa
catgaaaa
atgaaaat
tgaaaatg
gaaaatga
aaaatgaa
aaatgaac
aatgaact
atgaacta
tgaactaa
gaactaat
aactaata
actaatat
ctaatata
taatatac
aatatacc
atatacct
tatacctc
atacctcc
tacctcct
acctcctt
cctccttt
ctcctttc
tcctttct
cctttctt
ctttcttt
tttcttta
ttctttat
tctttata
ctttataa
tttataaa
ttataaat
tataaatt
ataaatta
taaattac
aaattact
aattactc
attactca
ttactcag
tactcagg
actcaggt
ctcaggta
tcaggtag
caggtagt
aggtagta
ggtagtat
gtagtatc
tagtatct
agtatctt
gtatcttt
tatcttta
atctttat
tctttata
ctttatag
tttatagc
ttatagca
tatagcag
atagcagt
tagcagtg
agcagtgt
gcagtgtg
cagtgtga
agtgtgag
gtgtgaga
tgtgagaa
gtgagaaa
tgagaaag
gagaaagg


263

In [31]:
# Now let's move everything to upper case

mpra_df['sequence_ancestral'] = mpra_df['sequence_ancestral'].str.upper()
mpra_df['sequence_derived'] = mpra_df['sequence_derived'].str.upper()

In [36]:
mpra_df.loc[12345:12350, ['sequence_derived', 'sequence_ancestral'] ]

,sequence_derived,sequence_ancestral
12345,TAATAGTTTAAATAGTGCCTTGGAGAGTAAGAAGATCTACTGTTAA...,TAATAGTTTAAATAGTGCCTTGGAGAGTAAGAAGATCTACTGTTAA...
12346,TCTTTGGAATGTCCTATCGTATTGCCAGGACAACACCTGAGTTTGT...,TCTTTGGAATGTCCTATCGTATTGCCAGGACAACACCTGAGTTTGT...
12347,CATAGGAGCTGGGGAGAGTGAGGGAAGAGTTGGGCTGGATTGGATC...,CATAGGAGCTGGGGAGAGTGAGGGAAGAGTTGGGCTGGATTGGATC...
12348,CAAACCACTAGGTTCTGCTGTCAGAAATCTCTGAGTCACGCTCACT...,CAAACCACTAGGTTCTGCTGTCAGAAATCTCTGAGTCACGCTCACT...
12349,GTTTCACCATGTTGGTCAGGCTGGTCTCGAACTCCTGACCTCGTGA...,GTTTCACCATGTTGGTCAGGCTGGTCTCGAACTCCTGACCTCGTGA...
12350,TCATCTCAGATATGCCAGGCTTAGAGCCTTTAAACTAACTGTCCAC...,TCATCTCAGATATGCCAGGCTTAGAGCCTTTAAACTAACTGTCCAC...


Here I will check which version of my reverese complement function is more efficient. 

In [48]:
import timeit

# Measure the execution time of um.get_reverse_complement
time_old = timeit.timeit(
    stmt="for seq in mpra_df['sequence_ancestral']: um.get_reverse_complement(seq)",
    globals=globals(),
    number=1  # Run the loop 10 times to get an average time
)

# Measure the execution time of um.new_get_reverse_complement
time_new = timeit.timeit(
    stmt="for seq in mpra_df['sequence_ancestral']: um.new_get_reverse_complement(seq)",
    globals=globals(),
    number=1  # Run the loop 10 times to get an average time
)

print(f"Old function time: {time_old:.6f} seconds")
print(f"New function time: {time_new:.6f} seconds")


Old function time: 11.726729 seconds
New function time: 0.347941 seconds


In [49]:
time_bulk = timeit.timeit(
    stmt="um.new_get_reverse_complement(mpra_df['sequence_ancestral'])",
    globals=globals(),
    number=1  # Run the loop 10 times to get an average time
)

print(f"Bulk function time: {time_bulk:.6f} seconds")

Bulk function time: 0.247965 seconds
